### PENDING:

1. Retain records when player out of game
2. Add: 
    - records of faced-off cards and revise prob of lie
    - when only 2 players remain, 1 player has 1 card remains, the other always bust despite of the style & risk
3. Make modular function for moves
4. Bootstrap
5. Data viz
6. Interactive with real human playing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Set up game:

In [3]:
moves = ["bust", "pass", "true", "lie"]

Set up player

In [4]:
player = np.array(["Thanos", "Ironman", "Batman", "Human"])
print("Number of player must be divided by 4") if len(player) % 4 != 0 else len(player)

4

Set up honesty level for each player. Honesty runs from 0 to 1, with 0 being always lie and 1 being always honest

In [5]:
honesty = np.random.rand(4)
honesty

array([0.25216926, 0.45539206, 0.62400813, 0.90250959])

### Set up style of player:

yolo: This player makes a move simply based on flipping a coin. If the coin tells him to play true while he does not have the card, he will just pass the move

safe: this player will only play true or pass. He nevers bust or lie to avoid the risk of picking up the cards.

thinker: this player analyses all players and keep track of faced-off cards, and consider risks and rewards before making a move

In [6]:
style = np.random.choice(["yolo", "thinker", "safe"], replace = True, size = 4)
style

array(['yolo', 'thinker', 'thinker', 'thinker'], dtype='<U7')

In [16]:
winners = []

Reward and penalty matrix for current player:
![table1.png](table1.png)

Flow of codes for each game: assemble deck, split cards to each player, play.

### Assign cards for each player

In [49]:
ranks = np.append(np.arange(2,11), ["A", "J", "Q", "K"])
deck = np.repeat(ranks, len(player))
card_num = len(deck)/len(player)
cards = []
for i in range(0,len(player)):
    idx = np.random.choice(range(0,len(deck)), replace = False, size = int(card_num))
    cards.append(np.sort(deck[idx]))
    deck = np.delete(deck, idx)

#Initiate round number:
round_num = 1

#Initiate placeholder for winner and runner-ups in a game:
winner = []

#Initiate placehoder for players whose fate are on hold:
round_winner = ""
anchor = None
busted_player = None

#Initiate tracking of picked up cards in row of 3+
r3_records = [[],[],[],[]]

#Initiate probability of lying and probability of bust
prob_lie = np.repeat(0.5, len(player))
prob_bust = np.repeat(0.5, len(player))

#Initiate placehoder for moves played
moves_records = np.zeros((4, len(player))) 
    #row 0 is times when lie is exposed, row 1 is number of place move 
    #row 2 is times when bust is made, row 3 is number of moves
    #number of columns is number of player

#Initiate tracking of safe cards played by round starter after each failed bust
safe_records = np.array([])
fail_bust = False

#Game stops when only 1 player remains:
player_num = len(player)
while player_num > 1:
    #For each round:    
    #initiate number of 'pass' move and initiate status of 'bust' move (move > 0 means no 'bust' happens yet)
    pass_num = 0

    #Initiate safe cards played by round starter after each failed bust
    #if fail_bust == True: safe_num = 0
    safe_num = 0
    
    move = 1
    
    #Initiate string of placed cards:
    place = []
    
    #First player of round 1 is random, of 2nd round onwards is the previous round's winner:
    first_player = np.random.choice(range(len(player))) if round_num == 1 else np.where(player == round_winner)[0][0]
    print("ROUND ", round_num)
    print("first player:", player[first_player])
    
    #First move of round is always to place a card:
    idx = np.random.choice(range(len(cards[first_player])))    
    place.append(cards[first_player][idx]) 
    cards[first_player] = np.delete(cards[first_player], idx)
    idx = np.random.choice(np.where(ranks != place[-1])[0])
    call = ranks[idx] if (honesty[first_player] < 0.5) else place[-1]
    moves_records[1,first_player] += 1
    moves_records[3,first_player] += 1
    print("place: ", place, "call: ", call)
    print("\t",cards[first_player])
    
    #Initiate string of players:
    previous_player = first_player
    remain = np.where(~np.isin(player, winner))[0]
    remain = np.roll(remain, len(remain)-np.where(remain == previous_player)[0][0])               
    current_player = remain[1]
    next_player = remain[2] if player_num >2 else remain[0]    
    
    #Round stops when all other players play 'pass' move sequentially or when there is a bust
    while (pass_num < (player_num - 1)) & (move != 0):
        #update probability of lie and bust
        if moves_records[1,previous_player] != 0:
            prob_lie[previous_player] = moves_records[0,previous_player]/moves_records[1,previous_player] 
        if moves_records[3,next_player] != 0:
            prob_bust[next_player] = moves_records[2,next_player]/moves_records[3,next_player]
        #average cards in row of 3 and average safe cards
        r3_num = np.nanmean(r3_records[current_player]) if len(r3_records[current_player]) !=0 else 0
        safe_avg = np.nanmean(safe_records) if len(safe_records) != 0 else 0
        
        #proportion of clearing a card: (1/current player's cards)*100
        cleared_prop = round(100/len(cards[current_player]),0)        
          
        #estimate rewards and penalty
        reward_bust = np.array([r3_num*(1-prob_lie[previous_player]), len(place)*prob_lie[previous_player], 0, 0])
        reward_pass = np.array([0,0, 0, 0])
        reward_true = np.array([0, 0, 
                                (cleared_prop + safe_avg)*prob_bust[next_player] if call in cards[current_player] else 0,
                                cleared_prop*(1-prob_bust[next_player]) if call in cards[current_player] else 0])
        reward_lie = np.array([0, 0, 
                               r3_num*prob_bust[next_player], 
                               cleared_prop*(1-prob_bust[next_player])])
        
        penalty_bust = np.array([len(place)*(1-prob_lie[previous_player]), 0, 0, 0])
        penalty_pass = np.array([0, 0, 0, 0])
        penalty_true = np.array([0, 0, 0, 0])
        penalty_lie = np.array([0, 0, len(place)*prob_bust[next_player], 0])
        
        net_reward = [reward_bust - penalty_bust, reward_pass - penalty_pass,
                      reward_true - penalty_true, reward_lie - penalty_lie]       
        
        #next player play a move:
        if style[current_player] == "yolo":
            move = np.random.choice(range(len(moves)))
            if move == 0: #bust
                moves_records[2,current_player] += 1 #update bust move to record
                temp = pd.Series(place).value_counts()
                if anchor is not None: busted_player = anchor  
                else: busted_player = previous_player                
                if place[-1] == call:
                    cards[current_player] = np.append(cards[current_player], place)
                    r3_records[current_player].append(temp[np.where(temp >= 3)[0]].sum())
                else: 
                    cards[busted_player] = np.append(cards[busted_player], place)
                    moves_records[0,busted_player] += 1 #update lie exposed to record
                    r3_records[busted_player].append(temp[np.where(temp >= 3)[0]].sum())                
                pass_num = 0           
            elif move == 1: #pass
                pass_num += 1
            elif move == 2: #true
                #if the desired card is not available, play 'pass' move:
                if call in cards[current_player]:
                    place.append(call)
                    cards[current_player] = np.delete(cards[current_player], 
                                                      np.where(cards[current_player] == call)[0][0])
                    pass_num = 0
                else:
                    pass_num += 1
                    move = 1
            else: #lie
                idx = np.random.choice(np.where(cards[current_player]!=call)[0])                
                place.append(cards[current_player][idx])
                cards[current_player] = np.delete(cards[current_player], idx)
                pass_num = 0
        elif style[current_player] == "thinker":
            move = np.where(net_reward == np.amax(net_reward))[0][0]
            if move == 0: #bust
                moves_records[2,current_player] += 1 #update bust move to record
                temp = pd.Series(place).value_counts()
                if anchor is not None: busted_player = anchor  
                else: busted_player = previous_player                
                if place[-1] == call:
                    cards[current_player] = np.append(cards[current_player], place)
                    r3_records[current_player].append(temp[np.where(temp >= 3)[0]].sum())
                else: 
                    cards[busted_player] = np.append(cards[busted_player], place)
                    moves_records[0,busted_player] += 1 #update lie exposed to record
                    r3_records[busted_player].append(temp[np.where(temp >= 3)[0]].sum())
                pass_num = 0 
            elif move == 1: #pass
                pass_num += 1
            elif move == 2: #true or pass
                #if the desired card is not available, play 'pass' move:
                if call in cards[current_player]: #true
                    place.append(call)
                    cards[current_player] = np.delete(cards[current_player], 
                                                      np.where(cards[current_player] == call)[0][0])
                    pass_num = 0
                else: #pass
                    pass_num += 1
                    move = 1
            else: #lie
                idx = np.random.choice(np.where(cards[current_player]!=call)[0])                
                place.append(cards[current_player][idx])
                cards[current_player] = np.delete(cards[current_player], idx)
                pass_num = 0
        else: #safe style
            #if the desired card is not available, play 'pass' move:
            if call in cards[current_player]: #true
                place.append(call)
                cards[current_player] = np.delete(cards[current_player], np.where(cards[current_player] == call)[0][0])
                pass_num = 0
                move = 2
            else: #pass
                pass_num += 1
                move = 1
        moves_records[3,current_player] += 1 #update current move to record
        print(player[current_player],"-",style[current_player],", plays", moves[move])
        print("place:", place)

        #anchor previous player whose fate is on hold
        if pass_num == 1: anchor = previous_player
        
        #Increment safe cards played by round starters after a failed bust
        if fail_bust == True:
            if anchor is not None:
                if ((pass_num == player_num - 1) | (move > 1)) & (anchor == first_player): safe_num += 1
            else:
                if (move > 1) & (previous_player == first_player): safe_num += 1
        
        if (move == 0) & (call == place[-1]): fail_bust = True
        if (move == 0) & (call != place[-1]): fail_bust = False
        
        #count cards of players remain in the round
        card_count = [len(i) for i in cards]

        #roll string of player before removing zero-card players
        remain = np.where(~np.isin(player, winner))[0]
        if (move == 0) & (place[-1] != call):
            remain = np.roll(remain, len(remain)-np.where(remain == busted_player)[0][0])
        elif move == 1:
            remain = np.roll(remain, len(remain)-np.where(remain == anchor)[0][0])
        else:
            remain = np.roll(remain, len(remain)-np.where(remain == previous_player)[0][0])        

        #Update the round winner:
        if move == 0:
            idx_win = busted_player if place[-1] == call else current_player
            if len(cards[idx_win]) == 0: idx_win = remain[1]  
            round_winner = player[idx_win]
        if pass_num == player_num - 1:
            if anchor is not None: idx_win = anchor
            if len(cards[idx_win]) == 0: idx_win = remain[1]
            round_winner = player[idx_win]
        
        print("\t",cards[current_player])
        print("\t player num: ", player_num, ", round winner: ", round_winner)
        print("\t anchor: ", player[anchor] if anchor is not None else "none", 
              ", busted: ", player[busted_player] if busted_player is not None else "none")        
        
        #remove player with 0 cards out of game
        if anchor is not None:
            if (card_count[anchor] == 0) & (pass_num == player_num - 1): winner.append(player[anchor])
            if (card_count[anchor] == 0) & (move > 1): winner.append(player[anchor])
            if (card_count[anchor] == 0) & (move == 0): winner.append(player[anchor])
            #reset anchor when his fate is no longer on hold
            if (player[anchor] in winner) | (move != 1) | (pass_num == player_num - 1): anchor = None
        if (move > 1) & (card_count[previous_player] == 0): winner.append(player[previous_player])
        if (move == 0) & (card_count[previous_player] == 0): winner.append(player[previous_player])
              
        player_num = len(player) - len(winner)        
        if player_num == 1: move == 0
        
        #increment of the position of player
        previous_player = current_player 
        remain = np.where(~np.isin(player, winner))[0]
        remain = np.roll(remain, len(remain)-np.where(remain == previous_player)[0][0])               
        current_player = remain[1] if player_num >1 else previous_player
        next_player = remain[2] if player_num >2 else remain[0]

        print("\t winner: ", winner, ", remain:", player[remain])
    
    #Update safe cards record:
    if safe_num > 0: safe_records = np.append(safe_records, safe_num)    
    
    #Update round number
    round_num += 1
    
#record winner of the game
winners.append(winner[0])

ROUND  1
first player: Ironman
place:  ['5'] call:  2
	 ['10' '2' '2' '2' '4' '7' '8' '9' 'A' 'J' 'K' 'Q']
Batman - thinker , plays lie
place: ['5', '6']
	 ['10' '10' '3' '3' '4' '6' '6' '7' '8' '9' 'J' 'Q']
	 player num:  4 , round winner:  
	 anchor:  none , busted:  none
	 winner:  [] , remain: ['Batman' 'Human' 'Thanos' 'Ironman']
Human - thinker , plays lie
place: ['5', '6', '3']
	 ['4' '5' '5' '6' '7' '9' '9' 'A' 'J' 'J' 'K' 'Q']
	 player num:  4 , round winner:  
	 anchor:  none , busted:  none
	 winner:  [] , remain: ['Human' 'Thanos' 'Ironman' 'Batman']
Thanos - yolo , plays lie
place: ['5', '6', '3', '3']
	 ['10' '2' '4' '5' '7' '8' '8' 'A' 'A' 'K' 'K' 'Q']
	 player num:  4 , round winner:  
	 anchor:  none , busted:  none
	 winner:  [] , remain: ['Thanos' 'Ironman' 'Batman' 'Human']
Ironman - thinker , plays true
place: ['5', '6', '3', '3', '2']
	 ['10' '2' '2' '4' '7' '8' '9' 'A' 'J' 'K' 'Q']
	 player num:  4 , round winner:  
	 anchor:  none , busted:  none
	 winner:  [] ,

	 winner:  ['Ironman'] , remain: ['Human' 'Thanos' 'Batman']
Thanos - yolo , plays bust
place: ['Q', 'Q', 'J', 'Q', '9', '10', '6']
	 ['2' '4' '5' '8' 'K' 'Q' 'K' '8' '6']
	 player num:  3 , round winner:  Thanos
	 anchor:  none , busted:  Human
	 winner:  ['Ironman'] , remain: ['Thanos' 'Batman' 'Human']
ROUND  9
first player: Thanos
place:  ['6'] call:  K
	 ['2' '4' '5' '8' 'K' 'Q' 'K' '8']
Batman - thinker , plays true
place: ['6', 'K']
	 ['3' '3' '4' '6' '4' '9' '7' '7' '7' '7' '2' 'J' '10' '10' 'A' 'A' 'A' 'A'
 '4' '2' '8' 'J' '8' '6' '2' '10' '3']
	 player num:  3 , round winner:  Thanos
	 anchor:  none , busted:  Human
	 winner:  ['Ironman'] , remain: ['Batman' 'Human' 'Thanos']
Human - thinker , plays true
place: ['6', 'K', 'K']
	 ['5' '9' '5' '3' '9' 'J' '5' 'Q' 'Q' 'J' 'Q' '9' '10' '6']
	 player num:  3 , round winner:  Thanos
	 anchor:  none , busted:  Human
	 winner:  ['Ironman'] , remain: ['Human' 'Thanos' 'Batman']
Thanos - yolo , plays lie
place: ['6', 'K', 'K', 'Q']
	 [

Batman - thinker , plays true
place: ['2', '3', '3', '3', 'Q', '3']
	 ['4' '6' '4' '9' '7' '7' 'A' 'A' 'A' '5' '5' '5' 'A' '8' '8' '8' '8' 'J'
 '7' '2' '4' '4']
	 player num:  3 , round winner:  Thanos
	 anchor:  Human , busted:  Human
	 winner:  ['Ironman'] , remain: ['Batman' 'Human' 'Thanos']
Human - thinker , plays lie
place: ['2', '3', '3', '3', 'Q', '3', '6']
	 ['9' 'Q' '6' '6' '7' '5' 'Q' 'J' 'J' 'J' 'Q' 'K' 'K' 'K' 'K' '9' '2' '10'
 '10' '10' '10' '9']
	 player num:  3 , round winner:  Thanos
	 anchor:  none , busted:  Human
	 winner:  ['Ironman'] , remain: ['Human' 'Thanos' 'Batman']
Thanos - yolo , plays lie
place: ['2', '3', '3', '3', 'Q', '3', '6', '2']
	 []
	 player num:  3 , round winner:  Thanos
	 anchor:  none , busted:  Human
	 winner:  ['Ironman'] , remain: ['Thanos' 'Batman' 'Human']
Batman - thinker , plays lie
place: ['2', '3', '3', '3', 'Q', '3', '6', '2', 'A']
	 ['4' '6' '4' '9' '7' '7' 'A' 'A' '5' '5' '5' 'A' '8' '8' '8' '8' 'J' '7'
 '2' '4' '4']
	 player num:  

	 player num:  2 , round winner:  Human
	 anchor:  none , busted:  Batman
	 winner:  ['Ironman', 'Thanos'] , remain: ['Human' 'Batman']
ROUND  39
first player: Human
place:  ['3'] call:  3
	 ['6' '7' 'K' 'K' '9' '10' '3' 'Q' '3' '6' '2' 'J' '5' 'K' '4' '5' '5' '5'
 '4' '4' '4' '10' '10' '10' 'J' 'J' 'J' 'A' 'A' 'A' 'A' '8' '3' '8' 'Q'
 'Q' 'Q']
Batman - thinker , plays lie
place: ['3', '7']
	 ['7' '8' '8' '7' '6' '2' '2' '6' 'K' '2' '9' '9' '9']
	 player num:  2 , round winner:  Human
	 anchor:  none , busted:  Batman
	 winner:  ['Ironman', 'Thanos'] , remain: ['Batman' 'Human']
Human - thinker , plays true
place: ['3', '7', '3']
	 ['6' '7' 'K' 'K' '9' '10' 'Q' '3' '6' '2' 'J' '5' 'K' '4' '5' '5' '5' '4'
 '4' '4' '10' '10' '10' 'J' 'J' 'J' 'A' 'A' 'A' 'A' '8' '3' '8' 'Q' 'Q'
 'Q']
	 player num:  2 , round winner:  Human
	 anchor:  none , busted:  Batman
	 winner:  ['Ironman', 'Thanos'] , remain: ['Human' 'Batman']
Batman - thinker , plays lie
place: ['3', '7', '3', '6']
	 ['7' '8' '8' 

	 anchor:  none , busted:  Batman
	 winner:  ['Ironman', 'Thanos'] , remain: ['Batman' 'Human']
Human - thinker , plays bust
place: ['10', '10', '5']
	 ['K' '6' '5' 'K' '4' '5' '5' '4' '4' '10' '10' 'J' 'J' 'A' 'A' 'A' '8' 'Q'
 '3' '3' '3' '3' '7' '7' '9' '2' '7' '7' '2' '2' '2' 'J' 'J' '6' '6' '6'
 '9' '9' '9' '8' '8' '8' 'K' 'K' 'Q' 'Q' 'Q']
	 player num:  2 , round winner:  Human
	 anchor:  none , busted:  Batman
	 winner:  ['Ironman', 'Thanos'] , remain: ['Human' 'Batman']
ROUND  62
first player: Human
place:  ['2'] call:  2
	 ['K' '6' '5' 'K' '4' '5' '5' '4' '4' '10' '10' 'J' 'J' 'A' 'A' 'A' '8' 'Q'
 '3' '3' '3' '3' '7' '7' '9' '2' '7' '7' '2' '2' 'J' 'J' '6' '6' '6' '9'
 '9' '9' '8' '8' '8' 'K' 'K' 'Q' 'Q' 'Q']
Batman - thinker , plays lie
place: ['2', '10']
	 ['4' 'A' '10' '5']
	 player num:  2 , round winner:  Human
	 anchor:  none , busted:  Batman
	 winner:  ['Ironman', 'Thanos'] , remain: ['Batman' 'Human']
Human - thinker , plays bust
place: ['2', '10']
	 ['K' '6' '5' 'K' '

	 player num:  2 , round winner:  Human
	 anchor:  none , busted:  Batman
	 winner:  ['Ironman', 'Thanos'] , remain: ['Batman' 'Human']
Human - thinker , plays bust
place: ['5', 'A']
	 ['5' '4' '10' 'J' 'J' 'A' 'A' 'A' 'Q' '3' '3' '7' '7' '9' '7' 'J' 'J' '6'
 '6' '9' '9' '9' 'Q' 'Q' 'Q' '2' '2' '8' '10' '10' '8' '4' '4' '4' '3' '3'
 'K' '8' '8' 'K' 'K' '5' '5']
	 player num:  2 , round winner:  Human
	 anchor:  none , busted:  Batman
	 winner:  ['Ironman', 'Thanos'] , remain: ['Human' 'Batman']
ROUND  90
first player: Human
place:  ['3'] call:  3
	 ['5' '4' '10' 'J' 'J' 'A' 'A' 'A' 'Q' '3' '3' '7' '7' '9' '7' 'J' 'J' '6'
 '6' '9' '9' '9' 'Q' 'Q' 'Q' '2' '2' '8' '10' '10' '8' '4' '4' '4' '3' 'K'
 '8' '8' 'K' 'K' '5' '5']
Batman - thinker , plays lie
place: ['3', 'K']
	 ['10' '2' '2' '6' '7' '6' '5' 'A']
	 player num:  2 , round winner:  Human
	 anchor:  none , busted:  Batman
	 winner:  ['Ironman', 'Thanos'] , remain: ['Batman' 'Human']
Human - thinker , plays bust
place: ['3', 'K']
	 [

	 winner:  ['Ironman', 'Thanos'] , remain: ['Human' 'Batman']
Batman - thinker , plays true
place: ['2', '2', '2']
	 []
	 player num:  2 , round winner:  Batman
	 anchor:  none , busted:  Batman
	 winner:  ['Ironman', 'Thanos'] , remain: ['Batman' 'Human']
Human - thinker , plays bust
place: ['2', '2', '2']
	 ['4' 'A' 'Q' '3' '7' '9' 'J' '6' '9' '9' '9' 'Q' 'Q' 'Q' '8' '10' '10' '4'
 '4' '4' '8' 'K' '5' '3' '2' '7' '10' '10' 'K' 'K' 'K' '6' '6' '6' 'A' 'A'
 'A' '7' '7' '8' '8' 'J' 'J' 'J' '5' '5' '5' '3' '3' '2' '2' '2']
	 player num:  2 , round winner:  Human
	 anchor:  none , busted:  Batman


True

	 winner:  ['Ironman', 'Thanos', 'Batman'] , remain: ['Human']


In [50]:
safe_records

array([2., 3., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [57]:
player
style
card_count
anchor
winner
player[remain]
winners
moves_records

array(['Thanos', 'Ironman', 'Batman', 'Human'], dtype='<U7')

array(['yolo', 'thinker', 'thinker', 'thinker'], dtype='<U7')

[0, 0, 0, 52]

['Ironman', 'Thanos', 'Batman']

array(['Human'], dtype='<U7')

['Batman',
 'Batman',
 'Batman',
 'Batman',
 'Ironman',
 'Batman',
 'Human',
 'Ironman',
 'Batman',
 'Ironman']

array([[  0.,   0.,  54.,  10.],
       [  9.,   1.,  46.,  57.],
       [ 11.,   0.,   2., 100.],
       [ 53.,  13., 191., 246.]])